In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tqdm
plt.style.use('ggplot')

In [ ]:
path = "C:/Users/hitesh/Learning/TechGiG/Sapient Lending/data/"

In [ ]:
data = pd.read_csv(path+'test.csv')
data.head()

In [ ]:
data.SK_ID_CURR.nunique()

In [ ]:
data.shape

In [ ]:
na_col = data.isna().sum().nonzero()

In [ ]:
def selection(df, col):
    select = []
    for i in col:
        select.append(df.iloc[:,i])
    return pd.concat(select)

In [ ]:
df = selection(data,na_col)
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum().sort_values()

In [ ]:
m1 = data.dropna()

In [ ]:
m1.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

In [ ]:
corr = m1.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(55, 35))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

### MainCode

In [ ]:
col_set = [ 'CNT_FAM_MEMBERS','AMT_ANNUITY','AMT_GOODS_PRICE','EXT_SOURCE_2','DEF_60_CNT_SOCIAL_CIRCLE',
 'OBS_60_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'NAME_TYPE_SUITE',
'AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_DAY' ,    
'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR',     
'EXT_SOURCE_3']   

In [ ]:
remove_col = []
for i in df.columns:
    if i not in col_set:
        remove_col.append(i)
print(remove_col)

In [ ]:
remove_col2 = []
for i in data.columns:
    if i not in remove_col:
        remove_col2.append(i)
print(remove_col2)

In [ ]:
# High points are missing 
data = data[remove_col2]

In [ ]:
data.shape

In [ ]:
datadata = data.copy()

In [ ]:
data.drop(columns=[ 'AMT_REQ_CREDIT_BUREAU_YEAR', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_MON',
'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_YEAR' ], axis = 1, inplace = True)

In [ ]:
data.shape

In [ ]:
df1 = data.dropna()

In [ ]:
df1.shape

In [ ]:
df1.info()

In [ ]:
part = df1.select_dtypes(include= 'object')

In [ ]:
part.shape

In [ ]:
train_small_with_dummies = pd.get_dummies(part, dtype ='int64')

In [ ]:
df1.drop(columns= part.columns, inplace= True)

In [ ]:
df1.shape

In [ ]:
df1 = df1.join(train_small_with_dummies)

In [ ]:
df1.shape

In [ ]:
df1.drop(columns = ['SK_ID_CURR'],inplace = True)

In [ ]:
df1.head()

### Model

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import  metrics
from sklearn.model_selection import cross_validate

In [ ]:
train = df1[:123745]
train.shape

In [ ]:
test = df1[123745:]
test.shape

In [ ]:
trainX = train.iloc[:,1:]
trainX.shape

In [ ]:
trainX.head()

In [ ]:
trainY = train.iloc[:,0]
trainY.shape

In [ ]:
testX = test.iloc[:,1:]
testX.shape

In [ ]:
testY = test.iloc[:,0]
testY.shape

In [ ]:
lgb_train = lgb.Dataset(trainX, trainY)
lgb_eval = lgb.Dataset(testX,testY, reference=lgb_train)

In [ ]:
trainY.value_counts()

In [ ]:
lgbm_params = {
    'boosting': 'dart',
    'application': 'binary',
    'learning_rate': 0.02,
    'min_data_in_leaf': 30,
    'num_leaves': 34,
    'max_depth': 8,
    'feature_fraction': 0.9497036,
    'scale_pos_weight': 10,
    'drop_rate': 0.02,
    'reg_alpha': 0.041545473,
    'reg_lambda': 0.0735294,
    'min_split_gain': 0.0222415,
    'min_child_weight': 39.3259775,
}


cv_results = lgb.cv(train_set=lgb_train,
                     params=lgbm_params,
                     nfold=5,
                     num_boost_round=1000,
                     early_stopping_rounds=200,
                     stratified=True,
                     verbose_eval=50,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best LGBM CV result = {}'.format(np.max(cv_results['auc-mean'])))

In [ ]:
clf = lgb.train(train_set=lgb_train,
                 params=lgbm_params,
                 num_boost_round=optimum_boost_rounds)
                 
""" Predict on test set and create submission """
y_pred = clf.predict(testX)

In [ ]:
# Plot importances
fig, (ax, ax1) = plt.subplots(1, 2, figsize=[24, 12])
lgb.plot_importance(clf, ax=ax, max_num_features=20, importance_type='split')
lgb.plot_importance(clf, ax=ax1, max_num_features=20, importance_type='gain')
ax.set_title('Importance by splits')
ax1.set_title('Importance by gain')
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def plotImp(model, X , num = 20):
    feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(),X.columns)), 
                               columns=['Value','Feature'])
    plt.figure(figsize=(40, 30))
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.show()
    
plotImp(clf,trainX,50)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(trainX.columns,clf.feature_importance())), 
                               columns=['Feature','Value'])

In [ ]:
feature_imp.sort_values(by="Value", ascending=False, inplace= True)
feature_imp.reset_index(inplace = True, drop= True)

In [ ]:
select = list(feature_imp[:30].Feature)

In [ ]:
select

In [ ]:
sns.kdeplot(data.AMT_REQ_CREDIT_BUREAU_YEAR)

In [ ]:
corr = df1[select].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(30, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

In [ ]:
naVal = pd.DataFrame(sorted(zip(data.columns,data.isna().sum())), 
                               columns=['Feature','Value'])

In [ ]:
naVal.sort_values(by="Value", ascending=False, inplace= True)
naVal.reset_index(inplace = True, drop= True)

In [ ]:
naVal[:20]

In [ ]:
data.shape

In [ ]:
m2 = data.drop(data[data.AMT_REQ_CREDIT_BUREAU_YEAR.isna()].index)

In [ ]:
m2.shape

In [ ]:
m2.isna().sum().sort_values()

In [ ]:
# TYPE 1

In [ ]:
r1 = m2.drop(columns= ['OCCUPATION_TYPE'], axis =1)

In [ ]:
r1.dropna(inplace= True)

In [ ]:
r1.shape

In [ ]:
r1.to_csv("Type1.csv", index= False)

In [ ]:
# TYPE 2

In [ ]:
r2 = m2.dropna()

In [ ]:
r2.shape

In [ ]:
r2.to_csv("Type2.csv", index= False)

In [ ]:
m2.reset_index(drop= True, inplace= True)

In [ ]:
m3 = m2.copy()

In [ ]:
m4 = m2.drop(m2[m2.EXT_SOURCE_3.isna()].index)

In [ ]:
m4.shape

In [ ]:
m5 =m4.drop(m4[m4.OBS_60_CNT_SOCIAL_CIRCLE.isna()].index)

In [ ]:
m5.shape

In [ ]:
m5.isna().sum().sort_values()

In [ ]:
""" Two options 
    1. delete OCCUPATION_TYPE as column and then drop remaining features
    2. Have OCCUPATION_TYPE but dropna()"""

In [ ]:
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=50,      
            early_stopping_rounds=10, 
            valid_sets=[lgb_train,lgb_eval],
            valid_names=['train', 'eval'])

In [ ]:
target = 'TARGET'

In [ ]:
def modelfit(alg, dtrain, dtest, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        xgtest = xgb.DMatrix(dtest[predictors].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
          metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['TARGET'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['TARGET'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['TARGET'], dtrain_predprob))
    
#     Predict on testing data:
    dtest_pred = alg.predict_proba(dtest[predictors])[:,1]
#    results = test_results.merge(dtest[['ID','predprob']], on='ID')
    print ('AUC Score (Test): %f' % metrics.roc_auc_score(dtest['TARGET'], dtest_pred ))
    

In [ ]:
predictors = [x for x in train.columns if x not in [target]]

In [ ]:
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=-1,
        scale_pos_weight=1,
        seed=27, verbose = True)
modelfit(xgb1, train, test, predictors)

In [ ]:
param_test1 = {
    'max_depth':[1,2,3],
    'learning_rate' : [0.05,0.1,0.2]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=100, max_depth=3,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', scale_pos_weight=1, seed= 26), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=-1, cv=5,verbose =1)
gsearch1.fit(train[predictors],train[target])

In [ ]:
gsearch1.best_score_

In [ ]:
gsearch1.best_params_

In [ ]:
gsearch1.best_estimator_